# Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd

In [73]:
from datetime import datetime, timedelta

In [102]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [101]:
aggRaitings = (ratings
               .groupby('userId')
               .agg({'rating':['count'],'timestamp':['min','max']})
               .reset_index()
              )

aggRaitings = aggRaitings[aggRaitings['rating']['count'] > 99]

In [85]:
def get_delta(row):
    return datetime.fromtimestamp(row['timestamp']['max']) - datetime.fromtimestamp(row['timestamp']['min'])

In [87]:
aggRaitings['life'] = aggRaitings.apply(get_delta, axis=1)
aggRaitings.head()

userId rating   timestamp                           life
           count         min         max                   
3       4    204   949778714   949982274    2 days 08:32:40
4       5    100  1163373044  1163375145    0 days 00:35:01
7       8    116  1154389340  1154474527    0 days 23:39:47
14     15   1700   997937239  1469330735 5455 days 22:38:16
16     17    363  1127468587  1127476640    0 days 02:14:13

In [100]:
aggRaitings.agg({'life':['mean']})

,life
mean,455 days 05:29:08.566539


# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента 

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [106]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [107]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [108]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [105]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
# client_base

In [121]:
joined = client_base[['client_id']].merge(rzd,how='outer').merge(auto,how='outer').merge(air,how='outer')
joined

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [122]:
joined = client_base.merge(rzd,how='outer').merge(auto,how='outer').merge(air,how='outer')
joined

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


# Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

* visits (user_id, source)
* purchases (user_id, category)
* geo (user_id, geo_list)

Имея достаточно большое количество данных, можно постоить отчет по территориям, где было покупок больше, а где меньше. А также, что чаще покупают в каждом районе, а что реже.

Таблица по районам
Колонки: "Количество посетителей", "Количество покупок", "Высоковостребованная категория", "Низковостребованная категория"
Строки: "Название района"